In [160]:
import clingo

In [161]:
inst1 = """
#program base.
e(2,1,100).
e(3,2,100).
e(1,3,100).
e(1,2,10).
e(2,3,10).
e(3,1,10).
"""
inst2 = inst1 + """
#program base.
e(3,4,10).
e(4,1,10).
e(4,3,100).
e(1,4,100).
"""

In [162]:
base = """
v(U) :- e(U,_,_).

time(0..max_tour_length).

{ visitNodeAtTime(V,T) : v(V), time(T) }.

% No gaps between visits
:- visitNodeAtTime(_, T2), time(T1), T1<T2, not visitNodeAtTime(_, T1).

% No 2 nodes at the same time
:- visitNodeAtTime(U,T), visitNodeAtTime(V,T), U<V.

% Visited nodes must be linked with edges, and all edges must be visited
visited(U,V) :- visitNodeAtTime(U,T), visitNodeAtTime(V,T+1).
:- visited(U,V), not e(U,V,_).
:- e(U,V,_), not visited(U,V), not visited(V,U).

% The tour must be closed, i.e. first and last vertex are the same (choose `1`).
tourLength(T) :- time(T), visitNodeAtTime(_,T), not visitNodeAtTime(_,T+1).
visitNodeAtTime(1,0).
visitNodeAtTime(1,T) :- tourLength(T).

% Penalize with weight for every visited edge.
:~ visitNodeAtTime(U,T), visitNodeAtTime(V,T+1), e(U,V,C).[C@1, T]

#show visitNodeAtTime/2.
"""

In [163]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0
ctl.add('base', ['max_tour_length'], base)
ctl.add('inst1', [], inst1)

print('\nGrounding...')
%time ctl.ground([('inst1', [])])
%time ctl.ground([('base', [clingo.parse_term('26')])])

print('\nSolving...\n')
%time ctl.solve(on_model=lambda m: print(m.cost, m, '\n'))


Grounding...
CPU times: user 392 µs, sys: 0 ns, total: 392 µs
Wall time: 322 µs
CPU times: user 455 µs, sys: 0 ns, total: 455 µs
Wall time: 459 µs

Solving...

[] visitNodeAtTime(1,0) 

CPU times: user 161 µs, sys: 0 ns, total: 161 µs
Wall time: 164 µs


SolveResult(5)

# Incremental guess and check

In [164]:
gac1 = """

#program base.

v(U) :- e(U,_,_).
v(V) :- e(_,V,_).
time(0..3).

#program guess.

1 { visitNodeAtTime(V,T) : v(V) } 1 :- time(T).

#program check.

% Visited nodes must be linked with edges, and all edges must be visited
%visited(U,V) :- visitNodeAtTime(U,T), visitNodeAtTime(V,T+1).
%:- visited(U,V), e(U,V,_).
%:- visited(V,U), e(U,V,_).

% visitNodeAtTime(1,0)
% visitNodeAtTime(3,1)
% visitNodeAtTime(1,2)
% visitNodeAtTime(1,3)

connected(T) :- e(U,V,_), visitNodeAtTime(U,T), visitNodeAtTime(V,T+1).
connected(T) :- e(V,U,_), visitNodeAtTime(U,T), visitNodeAtTime(V,T+1).
disconnected :- not connected(T), T=0..2.
tour :- visitNodeAtTime(1,0), visitNodeAtTime(1,3).
bad :- not tour.
bad :- disconnected.
:- not bad.

% Penalize with weight for every visited edge.
% :~ visitNodeAtTime(U,T), visitNodeAtTime(V,T+1), e(U,V,C).[C@1, T]

"""

In [165]:
gac1 = """

#program base.

v(U) :- e(U,_,_).
v(V) :- e(_,V,_).
time(0..4).

#program guess.

1 { visitNodeAtTime(V,T) : v(V) } 1 :- time(T).
visited(U,V) :- visitNodeAtTime(U,T), visitNodeAtTime(V,T+1).

#program check.


% Visited nodes must be linked with edges, and all edges must be visited
bad :- visited(U,V), not e(U,V,_).
bad :- e(U,V,_), not visited(U,V), not visited(V,U).
bad :- visited(1,1).

% The tour must be closed, i.e. first and last vertex are the same (choose `1`).
bad :- not visitNodeAtTime(1,0).
bad :- not visitNodeAtTime(1,4).

% Penalize with weight for every visited edge.
% :~ visitNodeAtTime(U,T), visitNodeAtTime(V,T+1), e(U,V,C).[C@1, T]

:- not bad.

"""

In [166]:
ctl = clingo.Control(("--models", "0"))

ctl.add("base", [], inst1)
ctl.add("base", [], gac1)
ctl.add("base", [], """
        #show disconnected/0.
        #show connected/1.
        #show visitNodeAtTime/2.
        """
        )
ctl.ground([("base", ()), ("guess", ()), ("check", ())])
ctl.solve(on_model=print)

visitNodeAtTime(2,2) visitNodeAtTime(3,0) visitNodeAtTime(1,1) visitNodeAtTime(1,3) visitNodeAtTime(1,4)
visitNodeAtTime(2,2) visitNodeAtTime(3,0) visitNodeAtTime(3,4) visitNodeAtTime(1,1) visitNodeAtTime(1,3)
visitNodeAtTime(2,2) visitNodeAtTime(2,4) visitNodeAtTime(3,0) visitNodeAtTime(1,1) visitNodeAtTime(1,3)
visitNodeAtTime(2,0) visitNodeAtTime(2,2) visitNodeAtTime(2,4) visitNodeAtTime(1,1) visitNodeAtTime(1,3)
visitNodeAtTime(2,0) visitNodeAtTime(2,2) visitNodeAtTime(1,1) visitNodeAtTime(1,3) visitNodeAtTime(1,4)
visitNodeAtTime(2,0) visitNodeAtTime(2,2) visitNodeAtTime(3,4) visitNodeAtTime(1,1) visitNodeAtTime(1,3)
visitNodeAtTime(2,2) visitNodeAtTime(3,4) visitNodeAtTime(1,0) visitNodeAtTime(1,1) visitNodeAtTime(1,3)
visitNodeAtTime(2,2) visitNodeAtTime(2,4) visitNodeAtTime(1,0) visitNodeAtTime(1,1) visitNodeAtTime(1,3)
visitNodeAtTime(2,2) visitNodeAtTime(1,0) visitNodeAtTime(1,1) visitNodeAtTime(1,3) visitNodeAtTime(1,4)
visitNodeAtTime(2,2) visitNodeAtTime(2,4) visitNodeAtTi

<block>:2:9-30: info: no atoms over signature occur in program:
  disconnected/0

<block>:3:9-27: info: no atoms over signature occur in program:
  connected/1



SolveResult(5)

In [172]:
from gac import Transformer, GACPropagator
from clingo import ast

ctl = clingo.Control(("--models", "0"))

check = []
with ast.ProgramBuilder(ctl) as builder:
    trans = Transformer(builder, check)
    for file in [inst1,
                 gac1,
                 "#show visitNodeAtTime/2. #show visited/2.",
                 ]:
        ast.parse_string(file, trans.add)
ctl.register_propagator(GACPropagator(check))

ctl.ground([("base", [])])
ctl.solve(on_model=print)

visitNodeAtTime(2,1) visitNodeAtTime(3,2) visitNodeAtTime(3,3) visitNodeAtTime(1,0) visitNodeAtTime(1,4) visited(1,2) visited(2,3) visited(3,3) visited(3,1)
visitNodeAtTime(2,1) visitNodeAtTime(3,3) visitNodeAtTime(1,0) visitNodeAtTime(1,2) visitNodeAtTime(1,4) visited(1,2) visited(1,3) visited(2,1) visited(3,1)
visitNodeAtTime(2,1) visitNodeAtTime(2,3) visitNodeAtTime(3,2) visitNodeAtTime(1,0) visitNodeAtTime(1,4) visited(3,2) visited(1,2) visited(2,3) visited(2,1)
visitNodeAtTime(2,1) visitNodeAtTime(2,3) visitNodeAtTime(1,0) visitNodeAtTime(1,2) visitNodeAtTime(1,4) visited(1,2) visited(2,1)
visitNodeAtTime(2,3) visitNodeAtTime(3,1) visitNodeAtTime(1,0) visitNodeAtTime(1,2) visitNodeAtTime(1,4) visited(1,2) visited(1,3) visited(2,1) visited(3,1)
visitNodeAtTime(2,3) visitNodeAtTime(3,1) visitNodeAtTime(3,2) visitNodeAtTime(1,0) visitNodeAtTime(1,4) visited(3,2) visited(3,3) visited(1,3) visited(2,1)


SolveResult(5)

In [168]:
gac2 = """
#program guess.

1 {a(1);a(2)}.

#program check.

:- not a(1).

"""

In [169]:
ctl = clingo.Control(("--models", "0"))

check = []
with ast.ProgramBuilder(ctl) as builder:
    trans = Transformer(builder, check)
    for file in (gac2,):
        ast.parse_string(file, trans.add)
ctl.register_propagator(GACPropagator(check))

ctl.ground([("base", [])])
ctl.solve(on_model=print)

a(2)


SolveResult(5)

In [170]:
gac3 = """

#program guess.

{a;b;c}.

#program check.

bad :- a.
bad :- not b.
:- not bad.

"""

In [171]:
ctl = clingo.Control(("--models", "0"))

check = []
with ast.ProgramBuilder(ctl) as builder:
    trans = Transformer(builder, check)
    for file in (gac3,):
        ast.parse_string(file, trans.add)
ctl.register_propagator(GACPropagator(check))

ctl.ground([("base", [])])
ctl.solve(on_model=print)

b
b c


SolveResult(5)